## 1. Models

In [1]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv


gpt4o = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # or your deployment
    api_version="2024-12-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...x
)

In [16]:
gpt41 = AzureChatOpenAI(
    azure_deployment="gpt-4.1",  # or your deployment
    api_version="2024-12-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...x
)

In [ ]:
resultat= gpt4o.invoke("capital of France ?")
resultat.content   

'The capital of France is **Paris**.'

In [ ]:
resultat= gpt41.invoke("capital of France ?")
resultat.content   

'The capital of France is **Paris**.'

## 2. Data ingestion 

### 2.1 Functions

In [3]:
import tiktoken
def count_token(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return print(f"Nombre de token: {num_tokens}") 

In [4]:
def save_docintel_tolocal_file(input_variable: str, nom_fichier_output: str): 
    with open(f"/Users/oussa/Desktop/Github_perso/fds/gen-fiches-synthese/doc_intel_output/{nom_fichier_output}.txt", "w") as text_file:
        text_file.write(input_variable)

In [11]:
def save_llm_ouput_tolocal_file(input_variable: str, nom_fichier_output: str, version: int): 
    with open(f"/Users/oussa/Desktop/Github_perso/fds/gen-fiches-synthese/llm_output/{nom_fichier_output}_version_{version}.txt", "w") as text_file:
        text_file.write(input_variable)

In [6]:
def mrkw_file_to_var(fichier_nom:str):
    with open(f"/Users/oussa/Desktop/Github_perso/fds/gen-fiches-synthese/doc_intel_output/{fichier_nom}.txt", "r") as f:
        resultat = f.read()
    return resultat

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, AnalyzeResult


def url_to_mrkdwn_docintel(url: str):
    """Convert URL file to markdown avec Document intelligence 

    Args:
        url (str): URL du fichier a convertir 

    Returns:
        str: Output content of the file
    """
    client = DocumentIntelligenceClient(
    endpoint=os.getenv("AZ_DOCINTEL_ENDPOINT") ,
    credential=AzureKeyCredential(os.getenv("AZ_DOCINTEL_KEY")),
    api_version="2024-11-30")
    
    poller = client.begin_analyze_document(
        "prebuilt-layout", AnalyzeDocumentRequest(url_source=url),  
        output_content_format="markdown"
    )
    receipts : AnalyzeResult = poller.result()
    return receipts.content

### 2.2 Apply

In [ ]:
# Cas 1: nouveau donc transformation + enregistrement : 
output_doctintel=url_to_mrkdwn_docintel("https://www.areas.fr/medias/fichiers/sfcr_areas_vie_2024.pdf")
save_docintel_tolocal_file(output_doctintel, "SFCR_areas_vie_2024_mrkdwn")
count_token(output_doctintel, "o200k_base")
# "https://www.areas.fr/medias/fichiers/sfcr_areas_dommages_2024.pdf"

In [8]:
# Cas 2: existe donc charge:
resultat_areas_vie_24=mrkw_file_to_var("SFCR_areas_vie_2024_mrkdwn")
resultat_areas_dmg_24=mrkw_file_to_var("SFCR_areas_dmg_2024_mrkwdn")
count_token(resultat_areas_dmg_24, "o200k_base")


Nombre de token: 79902


## 3 - llm output

### 3.1 - Method 1 (fenetre contextuelle)

In [ ]:
def fiche(input_variable_content:str , model): # gpt41 ou 
    SFCR_choisi=input_variable_content

    prompt = f"""Tu es un assistant expert dans le domaine financier et en assurance. 
    Ta tache : 
    - Extraire le maximum d informations à partir d'un texte tout en gardant du détail et les lier pour en faire un paragraphe qui correspond a chaque partie. 
    - Ces informations sont généralement des chiffres, des montants, des pourcentages, des tendances, des explications sur les changements et les évolutions, des dates sur les thématiques suivantes:
    
    -Présentation de l'organisme (tous les sujets relatif a : activités, histoire de création, nombre d'assurés, branche d'assurance,  secteurs d'activité, nombre d'assurés ou de sociétaires ).
    -Activité de l'organisme (tous les sujets relatif a :Chiffre d'affaires, primes, Résultat technique, Résultat financier, Résultat net après impôts).
    -Provisionnement (tous les sujets relatif a :  provisions techniques en valeur IFRS et Solvabilité II / 2 , la PREC ou provisions pour risques en cours des PSAP ou provisions pour sinistres a payer et des PPNA ou provisions pour primes non acquises ).
    -Solvabilité (tous les sujets relatif a : SCR, modèle interne , formule standard ,  capital de solvabilité requis, ratio de solvabilité , fonds propres , pourcentage et montants de répartition du SCR par catégorie ).
    -Placements autres que les actifs en represenation de contrats en UC ou indexe (tous les sujets relatif a : placements en valeur de marché , placements en valeur comptable, évolution de la valeur des placements , performances des placements, Répartition des placements par classe d'actifs).
    -Changement de la gouvernance (tous les sujets relatif a : Changement de gouvernance, nomination d'un dirigeant , nouveau mandat , renouvellement de mandat ).
    -Perspectives et vision de l'organisme et projets futurs et de développement (tous les sujets relatif a : Les projets futurs de développement, la stratégie ).

    #Règles: 
    - Tu ne dois pas modifier les valeurs des données financières (exemple: Chiffres d'affaires, pourcentages, taux, montants, ratio de solvabilité, ratio de couverture du SCR etc)
    - Tu t appuiyera exclusivement sur les informations dans le texte fourni pour extraire les indicateurs financièrs et les explications des évolutions de ces indicateurs.
    - Tu ne rajoutes pas d informations sur la base de tes connaissances
    - Tu t appuiyera au maximum sur les chiffres a chaque fois que tu donnes des informations. Si tu ne les trouves pas tu n inventes pas.
    - Pas la peine d'extraire les informations relatives a la conformité ( exemple a une directive ou au Règlement délégué )

    Fais cet exercice tel que je l'ai décrit pour le texte suivant:{SFCR_choisi}"""
    response = model.invoke(prompt)
    return response.content

In [ ]:
resultat= fiche(resultat_areas_dmg_24, gpt41)

"### Présentation de l'organisme\n\nAréas Dommages est une Société d'Assurance Mutuelle à cotisations fixes, régie par le Code des Assurances et immatriculée sous le numéro 775 670 466. Son siège social est situé au 47/49 rue de Miromesnil, 75008 Paris. Elle est la tête du groupe indépendant Aréas Assurances, qui constitue le 7ème réseau d'agents généraux en France. L'organisme exerce toutes les opérations d'assurance dommage en France, y compris dans les départements d'outre-mer et en Polynésie française. Le groupe inclut plusieurs entités, telles qu'Aréas Vie, Audassur SARL, et Gemma Participations SAS, couvrant des activités variées comme l'assurance vie, le courtage, et la gestion immobilière. En 2024, Aréas Dommages a généré un chiffre d'affaires de 641,6 millions d'euros, soit une croissance de 7,7 % par rapport à 2023. Les assurances automobiles et dommages aux biens représentent 68,2 % de l'activité.\n\n---\n\n### Activité de l'organisme\n\nLe chiffre d'affaires brut d'Aréas Do

In [ ]:
save_llm_ouput_tolocal_file(resultat, "fds_areas_dmg_41", 1)